In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip install dcase_util>=0.2.8
! pip install keras>=2.2.2
! pip install tensorflow>=1.9.0
! pip install sed_eval>=0.2.0
! pip install pretty_midi

In [ ]:
import tensorflow.compat.v1 as tf
import dcase_util
from dcase_util.containers import AudioContainer
import os, numpy
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
import sed_eval
%matplotlib inline
import os
import shutil # move files and delete folders with files
import tarfile
import urllib.request # download files folder
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
import pandas as pd
import IPython.display as ipd
import librosa, librosa.display
import re
import IPython # listen to sounds on Python

In [ ]:
# Handy tool to print data in HTML form
log = dcase_util.ui.FancyHTMLPrinter()

# Paths to store data
data_storage_path = '/content/data'
dataset_storage_path = os.path.join(data_storage_path, 'datasets')
feature_storage_path = os.path.join(data_storage_path, 'features_sed')
dcase_util.utils.Path().create(
    [data_storage_path, dataset_storage_path, feature_storage_path]
)

# Filename for acoustic model
model_filename = os.path.join(data_storage_path, 'model_sed.h5')  


dcase_util.keras.setup_keras(backend='tensorflow')
import keras
from keras.layers import *
from keras.models import Model

In [ ]:
def readAudio(filename):
    x, sr = librosa.load(filename, sr=16000)
    return x, sr

#calculate spectrogram
def calc_spec(x):
    n_fft = 1024
    hop_length = 512
    win_length = 1024
    X = np.abs(librosa.stft(x, n_fft = n_fft, hop_length = hop_length, win_length = win_length, window='hann', dtype = np.complex256))
    X = librosa.power_to_db(X**2,ref=np.max)
    return X

In [ ]:

def get_feature_matrix(audio_filename_path):  
    """Extract acoustic features (log mel-energies) for given audio file and store them."""
    
    audio, fs = librosa.load(audio_filename_path)
    hop_len = 0.02
    mel_extractor = dcase_util.features.MelExtractor(n_mels=40, win_length_seconds=0.04, hop_length_seconds=hop_len, fs = fs)
    mel_data = mel_extractor.extract(y=audio)
    #numpy.save(feature_filename, mel_data)
    return mel_data, hop_len
        

In [ ]:
def get_feature_matrix_script_features(audio_filename_path):  
    """Extract acoustic features (log mel-energies) for given audio file and store them."""
    
    audio, fs = librosa.load(audio_filename_path, sr = 16000)
    mel_data = calc_spec(audio)
    # hop_len = 0.02
    # mel_extractor = dcase_util.features.MelExtractor(n_mels=40, win_length_seconds=0.04, hop_length_seconds=hop_len, fs = fs)
    # mel_data = mel_extractor.extract(y=audio)
    #numpy.save(feature_filename, mel_data)

    return mel_data, 10/mel_data.shape[1]
        

In [ ]:
def csv_to_meta_container(csv_file_path):
  dict_container = dcase_util.containers.ListDictContainer(filename = csv_file_path)
  dict_container.load()
  train_meta = dcase_util.containers.MetaDataContainer(dict_container)
  print(train_meta)
  return train_meta

In [ ]:
train_meta = csv_to_meta_container('/content/drive/MyDrive/Ee603_mlsp/Project/dataset_updated/labels_updated.csv')

MetaDataContainer :: Class
Items                               : 2160 
Unique
  Files                             : 1020 
  Scene labels                      : 0 
  Event labels                      : 2 
  Tags                              : 0 
  Identifiers                       : 0 
  Source labels                     : 0 

Meta data
  Source                  Onset   Offset   Scene             Event             Tags              Identifier   
  --------------------   ------   ------   ---------------   ---------------   ---------------   -----   
  S1                       0.00     5.00   -                 music             -                 -       
  S1                       5.00    10.00   -                 speech            -                 -       
  S2                       0.00     5.00   -                 music             -                 -       
  S2                       5.00    10.00   -                 speech            -                 -       
  S3                 

In [ ]:
X_train_ = []
Y_train_ = []
audio_folder_name = '/content/drive/MyDrive/Ee603_mlsp/Project/dataset_updated/wav_updated'
for audio_filename in os.listdir(audio_folder_name):#os.listdir() s1.wav, s2.wav
    #print('Load', db.absolute_to_relative_path(audio_filename))
    
    # Extract features, load them from file if they exists, if not extract and save
    audio_path = os.path.join(audio_folder_name, audio_filename)
    features, hop_length_seconds = get_feature_matrix_script_features(audio_path)
   
    # Targets
    event_list = train_meta.filter(filename=audio_filename.split('.')[0])
    event_roll = event_list.to_event_roll(
        label_list = ['speech', 'music'],                                       # Event labels
        time_resolution = hop_length_seconds,                   # Time resolution of feature matrix
        length_seconds = features.shape[1] * hop_length_seconds   # Length of original audio signal
    ).data

    event_roll = np.array(event_roll)
    silence_roll = np.zeros((1, event_roll.shape[1]))
    for index_ in range(event_roll.shape[1]):
      if(event_roll[0][index_] < 0.5 and event_roll[1][index_] < 0.5):
        silence_roll[0][index_] = 1


    #stack silence roll
    event_roll = np.concatenate((event_roll,silence_roll), axis = 0)

    X_train_.append(features) 
    Y_train_.append(event_roll)

  


In [ ]:
# X_train_ = np.stack(X_train_)
# Y_train_ = np.stack(Y_train_)
Y_train_ = np.moveaxis(Y_train_, 1, 2)
print('----------------------')
print('X_train shape', X_train_.shape)
print('Y_train shape', Y_train_.shape)

----------------------
X_train shape (1008, 513, 313)
Y_train shape (1008, 313, 3)


In [ ]:
X_train = np.moveaxis(X_train_, 1, 2)
X_train = X_train.reshape(-1, X_train.shape[-1])
Y_train = Y_train_.reshape(-1, Y_train_.shape[-1])
print('X_train shape', X_train.shape)
print('Y_train shape', Y_train.shape)

X_train shape (315504, 513)
Y_train shape (315504, 3)


In [ ]:
music_count=0
speech_count=0
silence_count=0

for i in range(Y_train_.shape[0]):
  if(Y_train_[i][0][0]==1):
    speech_count = speech_count+1
  elif(Y_train_[i][0][1]==1):
    music_count = music_count+1
  else:
    silence_count = silence_count+1

total_count = Y_train_.shape[0]
music_prob = float(music_count)/total_count
speech_prob = float(speech_count)/total_count
silence_prob = float(silence_count)/total_count

initial_state_probs = pd.Series([music_prob,speech_prob,silence_prob], index=['music', 'speech','silence'])
display(initial_state_probs)

music      0.236111
speech     0.380952
silence    0.382937
dtype: float64

In [ ]:
def normalize(a, axis=None):
    """
    Normalize the input array so that it sums to 1.
    Parameters
    ----------
    a : array
        Non-normalized input data.
    axis : int
        Dimension along which normalization is performed.
    Notes
    -----
    Modifies the input **inplace**.
    """
    a_sum = a.sum(axis)
    if axis and a.ndim > 1:
        # Make sure we don't divide by zero.
        a_sum[a_sum == 0] = 1
        shape = list(a.shape)
        shape[axis] = 1
        a_sum.shape = shape

    a /= a_sum


In [ ]:
def log_normalize(a, axis=None):
    """
    Normalize the input array so that ``sum(exp(a)) == 1``.
    Parameters
    ----------
    a : array
        Non-normalized input data.
    axis : int
        Dimension along which normalization is performed.
    Notes
    -----
    Modifies the input **inplace**.
    """
    if axis is not None and a.shape[axis] == 1:
        # Handle single-state GMMHMM in the degenerate case normalizing a
        # single -inf to zero.
        a[:] = 0
    else:
        with np.errstate(under="ignore"):
            a_lse = special.logsumexp(a, axis, keepdims=True)
        a -= a_lse

    

In [ ]:
def iter_from_X_lengths(X, lengths):
    warnings.warn(
        "iter_from_X_lengths is deprecated and will be removed in the future.",
        DeprecationWarning, stacklevel=2)
    if lengths is None:
        yield 0, len(X)
    else:
        n_samples = X.shape[0]
        end = np.cumsum(lengths).astype(np.int32)
        start = end - lengths
        if end[-1] > n_samples:
            raise ValueError("more than {:d} samples in lengths array {!s}"
                             .format(n_samples, lengths))

        for i in range(len(lengths)):
            yield start[i], end[i]

In [ ]:

def log_mask_zero(a):
    """
    Compute the log of input probabilities masking divide by zero in log.
    Notes
    -----
    During the M-step of EM-algorithm, very small intermediate start
    or transition probabilities could be normalized to zero, causing a
    *RuntimeWarning: divide by zero encountered in log*.
    This function masks this unharmful warning.
    """
    a = np.asarray(a)
    with np.errstate(divide="ignore"):
        return np.log(a)


def fill_covars(covars, , n_components=1, n_features=1):
  return covars

Multivariate Normal Distribution Function

In [ ]:
def _log_multivariate_normal_density(X, means, covars, min_covar=1.e-7):
    """Log probability for full covariance matrices."""
    n_samples, n_dim = X.shape
    nmix = len(means)
    log_prob = np.empty((n_samples, nmix))
    for c, (mu, cv) in enumerate(zip(means, covars)):
        try:
            cv_chol = linalg.cholesky(cv, lower=True)
        except linalg.LinAlgError:
            # The model is most probably stuck in a component with too
            # few observations, we need to reinitialize this components
            try:
                cv_chol = linalg.cholesky(cv + min_covar * np.eye(n_dim),lower=True)
            except linalg.LinAlgError:
                raise ValueError("'covars' must be symmetric, ""positive-definite")

        cv_log_det = 2 * np.sum(np.log(np.diagonal(cv_chol)))
        cv_sol = linalg.solve_triangular(cv_chol, (X - mu).T, lower=True).T
        log_prob[:, c] = - .5 * (np.sum(cv_sol ** 2, axis=1) +  n_dim * np.log(2 * np.pi) + cv_log_det)

    return log_prob

Viterbi Algorithm

In [ ]:
def _viterbi(int n_samples, int n_components,
             dtype_t[:] log_startprob,
             dtype_t[:, :] log_transmat,
             dtype_t[:, :] framelogprob):

    cdef int i, j, t, where_from
    cdef dtype_t logprob

    cdef int[::view.contiguous] state_sequence = \
        np.empty(n_samples, dtype=np.int32)
    cdef dtype_t[:, ::view.contiguous] viterbi_lattice = \
        np.zeros((n_samples, n_components))
    cdef dtype_t[::view.contiguous] work_buffer = np.empty(n_components)

    with nogil:
        for i in range(n_components):
            viterbi_lattice[0, i] = log_startprob[i] + framelogprob[0, i]

        # Induction
        for t in range(1, n_samples):
            for i in range(n_components):
                for j in range(n_components):
                    work_buffer[j] = (log_transmat[j, i]
                                      + viterbi_lattice[t - 1, j])

                viterbi_lattice[t, i] = _max(work_buffer) + framelogprob[t, i]

        # Observation traceback
        state_sequence[n_samples - 1] = where_from = \
            _argmax(viterbi_lattice[n_samples - 1])
        logprob = viterbi_lattice[n_samples - 1, where_from]

        for t in range(n_samples - 2, -1, -1):
            for i in range(n_components):
                work_buffer[i] = (viterbi_lattice[t, i]+ log_transmat[i, where_from])

            state_sequence[t] = where_from = _argmax(work_buffer)

    return np.asarray(state_sequence), logprob

In [ ]:
class _HMM():
    """
    Base class for Hidden Markov Models.

    startprob_ : array, shape (n_components, )
        Initial state occupation distribution.
    transmat_ : array, shape (n_components, n_components)
        Matrix of transition probabilities between states.
    """

    def __init__(self, n_components=1,):
        """
        Parameters
        ----------
        n_components : int
            Number of states in the model.
        """
        self.n_components = n_components

    def get_stationary_distribution(self):
        """Compute the stationary distribution of states."""
        # The stationary distribution is proportional to the left-eigenvector
        # associated with the largest eigenvalue (i.e., 1) of the transition
        # matrix.
        check_is_fitted(self, "transmat_")
        eigvals, eigvecs = linalg.eig(self.transmat_.T)
        eigvec = np.real_if_close(eigvecs[:, np.argmax(eigvals)])
        return eigvec / eigvec.sum()

    def score_samples(self, X, lengths=None):
       """
         Compute the log probability under the model and compute posteriors.
        """
        return self._score(X, lengths, compute_posteriors=True)

    def score(self, X, lengths=None):
        """
        Compute the log probability under the model.
        Parameters
        ----------
      
        """
        return self._score(X, lengths, compute_posteriors=False)[0]

    def _score(self, X, lengths=None, *, compute_posteriors):
        """
        Helper for `score` and `score_samples`.
        Compute the log probability under the model, as well as posteriors if
        *compute_posteriors* is True (otherwise, an empty array is returned
        for the latter).
        """
        check_is_fitted(self, "startprob_")
        self._check()

        X = check_array(X)
        impl = {
            "scaling": self._score_scaling,
            "log": self._score_log,
        }[self.implementation]
        return impl(X=X, lengths=lengths, compute_posteriors=compute_posteriors)

    def _score_log(self, X, lengths=None, *, compute_posteriors):
        """
        Compute the log probability under the model, as well as posteriors if
        *compute_posteriors* is True (otherwise, an empty array is returned
        for the latter).
        """
        logprob = 0
        sub_posteriors = [np.empty((0, self.n_components))]
        for sub_X in split_X_lengths(X, lengths):
            framelogprob = self._compute_log_likelihood(sub_X)
            logprobij, fwdlattice = self._do_forward_log_pass(framelogprob)
            logprob += logprobij
            if compute_posteriors:
                bwdlattice = self._do_backward_log_pass(framelogprob)
                sub_posteriors.append(
                    self._compute_posteriors_log(fwdlattice, bwdlattice))
        return logprob, np.concatenate(sub_posteriors)

    def _score_scaling(self, X, lengths=None, *, compute_posteriors):
        logprob = 0
        sub_posteriors = [np.empty((0, self.n_components))]
        for sub_X in split_X_lengths(X, lengths):
            frameprob = self._compute_likelihood(sub_X)
            logprobij, fwdlattice, scaling_factors = \
                    self._do_forward_scaling_pass(frameprob)
            logprob += logprobij
            if compute_posteriors:
                bwdlattice = self._do_backward_scaling_pass(
                    frameprob, scaling_factors)
                sub_posteriors.append(
                    self._compute_posteriors_scaling(fwdlattice, bwdlattice))

        return logprob, np.concatenate(sub_posteriors)

    def _decode_viterbi(self, X):
        framelogprob = self._compute_log_likelihood(X)
        return self._do_viterbi_pass(framelogprob)

    def _decode_map(self, X):
        _, posteriors = self.score_samples(X)
        logprob = np.max(posteriors, axis=1).sum()
        state_sequence = np.argmax(posteriors, axis=1)
        return logprob, state_sequence

    def decode(self, X, lengths=None):
        """
          Find most likely state sequence corresponding to ``X``.
        """
        check_is_fitted(self, "startprob_")
        self._check()

        algorithm = algorithm or self.algorithm
        if algorithm not in DECODER_ALGORITHMS:
            raise ValueError("Unknown decoder {!r}".format(algorithm))

        decoder = {
            "viterbi": self._decode_viterbi,
            "map": self._decode_map
        }[algorithm]

        X = check_array(X)
        logprob = 0
        sub_state_sequences = []
        for sub_X in split_X_lengths(X, lengths):
            # XXX decoder works on a single sample at a time!
            sub_logprob, sub_state_sequence = decoder(sub_X)
            logprob += sub_logprob
            sub_state_sequences.append(sub_state_sequence)

        return logprob, np.concatenate(sub_state_sequences)

    def predict(self, X, lengths=None):
        """
        Find most likely state sequence corresponding to ``X``.
        Parameters
        ----------
        X : array-like, shape (n_samples, n_features)
            Feature matrix of individual samples.
        lengths : array-like of integers, shape (n_sequences, ), optional
            Lengths of the individual sequences in ``X``. The sum of
            these should be ``n_samples``.
        Returns
        -------
        state_sequence : array, shape (n_samples, )
            Labels for each sample from ``X``.
        """
        _, state_sequence = self.decode(X, lengths)
        return state_sequence

    def predict_proba(self, X, lengths=None):
        """
        Compute the posterior probability for each state in the model.
        Parameters
        ----------
        X : array-like, shape (n_samples, n_features)
            Feature matrix of individual samples.
        lengths : array-like of integers, shape (n_sequences, ), optional
            Lengths of the individual sequences in ``X``. The sum of
            these should be ``n_samples``.
        Returns
        -------
        posteriors : array, shape (n_samples, n_components)
            State-membership probabilities for each sample from ``X``.
        """
        _, posteriors = self.score_samples(X, lengths)
        return posteriors 


    def _do_viterbi_pass(self, framelogprob):
        n_samples, n_components = framelogprob.shape
        state_sequence, logprob = _viterbi(
            n_samples, n_components, log_mask_zero(self.startprob_),
            log_mask_zero(self.transmat_), framelogprob)
        return logprob, state_sequence


In [ ]:
class GMMM_HMM(_HMM):
    """
    Hidden Markov Model with Gaussian emissions.
    Attributes
    ----------
    n_features : int
        Dimensionality of the Gaussian emissions.
    monitor_ : ConvergenceMonitor
        Monitor object used to check the convergence of EM.
    startprob_ : array, shape (n_components, )
        Initial state occupation distribution.
    transmat_ : array, shape (n_components, n_components)
        Matrix of transition probabilities between states.
    means_ : array, shape (n_components, n_features)
        Mean parameters for each state.
    covars_ : array
        Covariance parameters for each state.
    """

    def __init__(self, n_components=1,
                 min_covar=1e-3,
                 startprob_prior=1.0, transmat_prior=1.0,
                 means_prior=0, means_weight=0,
                 covars_prior=1e-2, covars_weight=1,
                 algorithm="viterbi", random_state=None,
                 n_iter=10, tol=1e-2, verbose=False,
                 params="stmc", init_params="stmc",
                 implementation="log"):
        """
        Parameters
        ----------
        n_components : int
            Number of states.
        """
        _HMM.__init__(self, n_components)
        

    @property
    def covars_(self):
        """Return covars as a full matrix."""
        return fill_covars(self._covars_,
                           self.n_components, self.n_features)

    @covars_.setter
    def covars_(self, covars):
        covars = np.array(covars, copy=True)
        _validate_covars(covars,self.n_components)
        self._covars_ = covars

    def _get_n_fit_scalars_per_param(self):
        nc = self.n_components
        nf = self.n_features
        return {
            "s": nc - 1,
            "t": nc * (nc - 1),
            "m": nc * nf,
            "c": {
                "spherical": nc,
                "diag": nc * nf,
                "full": nc * nf * (nf + 1) // 2,
                "tied": nf * (nf + 1) // 2,
            }[self.covariance_type],
        }

    def _init(self, X, lengths=None):
        _check_and_set_gaussian_n_features(self, X)
        super()._init(X, lengths=lengths)

        if self._needs_init("m", "means_"):
            kmeans = cluster.KMeans(n_clusters=self.n_components,
                                    random_state=self.random_state)
            kmeans.fit(X)
            self.means_ = kmeans.cluster_centers_
        if self._needs_init("c", "covars_"):
            cv = np.cov(X.T) + self.min_covar * np.eye(X.shape[1])
            if not cv.shape:
                cv.shape = (1, 1)
            self.covars_ = \
                distribute_covar_matrix_to_match_covariance_type(
                    cv, self.n_components).copy()

    def _check(self):
        super()._check()

        self.means_ = np.asarray(self.means_)
        self.n_features = self.means_.shape[1]

        if self.covariance_type not in COVARIANCE_TYPES:
            raise ValueError('covariance_type must be one of {}'
                             .format(COVARIANCE_TYPES))

    def _compute_log_likelihood(self, X):
        return log_multivariate_normal_density(
            X, self.means_, self._covars_)

    def _generate_sample_from_state(self, state, random_state=None):
        random_state = check_random_state(random_state)
        return random_state.multivariate_normal(
            self.means_[state], self.covars_[state]
        )

    def _initialize_sufficient_statistics(self):
        stats = super()._initialize_sufficient_statistics()
        stats['post'] = np.zeros(self.n_components)
        stats['obs'] = np.zeros((self.n_components, self.n_features))
        stats['obs**2'] = np.zeros((self.n_components, self.n_features))
        if self.covariance_type in ('tied', 'full'):
            stats['obs*obs.T'] = np.zeros((self.n_components, self.n_features,
                                           self.n_features))
        return stats

    def _accumulate_sufficient_statistics(self, stats, obs, lattice,
                                          posteriors, fwdlattice, bwdlattice):
        super()._accumulate_sufficient_statistics(
            stats, obs, lattice, posteriors, fwdlattice, bwdlattice)

        if 'm' in self.params or 'c' in self.params:
            stats['post'] += posteriors.sum(axis=0)
            stats['obs'] += np.dot(posteriors.T, obs)

        if 'c' in self.params:
            if self.covariance_type in ('spherical', 'diag'):
                stats['obs**2'] += np.dot(posteriors.T, obs ** 2)
            elif self.covariance_type in ('tied', 'full'):
                # posteriors: (nt, nc); obs: (nt, nf); obs: (nt, nf)
                # -> (nc, nf, nf)
                stats['obs*obs.T'] += np.einsum(
                    'ij,ik,il->jkl', posteriors, obs, obs)

    def _do_mstep(self, stats):
        super()._do_mstep(stats)

        means_prior = self.means_prior
        means_weight = self.means_weight

       
        denom = stats['post'][:, None]
        if 'm' in self.params:
            self.means_ = ((means_weight * means_prior + stats['obs'])
                           / (means_weight + denom))

        if 'c' in self.params:
            covars_prior = self.covars_prior
            covars_weight = self.covars_weight
            meandiff = self.means_ - means_prior

              c_n = np.empty((self.n_components, self.n_features,
                              self.n_features))
              for c in range(self.n_components):
                  obsmean = np.outer(stats['obs'][c], self.means_[c])
                  c_n[c] = (means_weight * np.outer(meandiff[c],
                                                    meandiff[c])
                            + stats['obs*obs.T'][c]
                            - obsmean - obsmean.T
                            + np.outer(self.means_[c], self.means_[c])
                            * stats['post'][c])
              cvweight = max(covars_weight - self.n_features, 0)

Make utility functions for feature extraction, calculation of matrices and post processing

In [ ]:
COL_NAMES_FEATURES = []
for i in range(0,513):  #column headers
  COL_NAMES_FEATURES.append(str(i)+"c")
def __calc_prob_class(class_group):
    class_group_count = class_group.groupby('sequence_class').size().reset_index()
    class_group_count.columns = ['sequence_class', 'count']
    total = class_group_count['count'].sum()
    class_group_count['transition_prob'] = class_group_count['count']/total
    return class_group_count

Calulation of state transition probability matrix

In [ ]:
def calc_transition_prob_matrix(classes_annotation, test_version=False):
    initial_classes = classes_annotation['class'].values[:-1]
    sequence_class = classes_annotation['class'][1:].tolist()

    sequence_classes = pd.DataFrame({'initial_classes': initial_classes, 'sequence_class': sequence_class})
    prob_matrix = sequence_classes.groupby('initial_classes').apply(__calc_prob_class).reset_index().drop('level_1', axis=1)
    prob_matrix = prob_matrix.pivot(index='initial_classes', columns='sequence_class', values='transition_prob')
    prob_matrix.append(pd.Series(np.zeros(prob_matrix.shape[1]), name='<END>'))
    prob_matrix = prob_matrix.fillna(0)
    if(test_version == False):
        prob_matrix['<START>'] = 0.
        prob_matrix.loc['<END>'] = 0.
        prob_matrix.loc['<END>', '<END>'] = 1
    return prob_matrix

Calculate mean vector of GMM

In [ ]:
def __get_mu_array(note_feature_vector):
    return note_feature_vector[COL_NAMES_FEATURES].mean()

Calculate state covariance matrix

In [ ]:
def get_mu_sigma_from_spectro(spectrogram):
    mu_array = spectrogram.groupby('class').apply(__get_mu_array)

    states_cov_matrices = []
    for name, group in spectrogram.groupby('class'): # alphabetic order
        states_cov_matrices.append(group[COL_NAMES_FEATURES].cov().values)
    states_cov_matrices = np.array(states_cov_matrices)

    return [mu_array, states_cov_matrices]

In [ ]:
def get_hmm_predictions(class_ix_predictions, ix_2_class):
    return np.array([ix_2_class[class_ix] for class_ix in class_ix_predictions])

Functions for simplification of labels

In [ ]:
def __simplify_classes(classes_df):
    classes_processed = classes_df['class'].str.split(':maj')                         # remove major x classes return array of array
    classes_processed = [elem[0] for elem in classes_processed]                       # further process step above to return 1 array
    classes_processed = [elem.split('/')[0] for elem in classes_processed]            # remove inverted classes
    classes_processed = [elem.split('aug')[0] for elem in classes_processed]          # remove augmented classes
    classes_processed = [elem.split(':(')[0] for elem in classes_processed]           # remove added notes classes
    classes_processed = [elem.split('(')[0] for elem in classes_processed]            # remove added notes classes 2
    classes_processed = [elem.split(':sus')[0] for elem in classes_processed]         # remove sustained classes
    classes_processed = [re.split(":?\d", elem)[0] for elem in classes_processed]     # remove added note
    classes_processed = [elem.replace('dim', 'min') for elem in classes_processed]    # change diminute to minor
    classes_processed = [elem.replace('hmin', 'min') for elem in classes_processed]   # change semi-diminute to minor
    classes_processed = [re.split(":$", elem)[0] for elem in classes_processed]       # remove added notes classes
    return classes_processed

In [ ]:

def read_simplify_class_file(music_file_path, process_silence=False):
    classes_annotation = pd.read_csv(music_file_path, sep=" ", header=None)
    classes_annotation.columns = ['start', 'end', 'class']
    classes_annotation['class'] = __simplify_classes(classes_annotation)
    if(process_silence == True): # replace silence by probable tonal end
        classes_annotation.loc[classes_annotation['class'] == 'N', 'class'] = classes_annotation['class'].mode()[0]
    return classes_annotation


In [ ]:

def simplify_predicted_classes(spectrogram, predicted_col='predicted'):
    change_class = spectrogram[predicted_col] != spectrogram[predicted_col].shift(-1)
    change_class_ix = change_class[change_class == True].index
    filtered_pcp = spectrogram.loc[change_class_ix].copy()
    end_time_previous = np.array([0] + filtered_pcp['end'][:-1].tolist())
    filtered_pcp['start'] = end_time_previous
    return filtered_pcp[[predicted_col, 'start', 'end']].reset_index(drop=True)

#Spectrogram Feature Extraction

In [ ]:
def calc_spectrogram(x, Fs, plot=True):
    """ Calculates the spectrogram features of the audio file and outputs a numpy matrix of shape"""
    n_fft = 1024
    hop_length = 512
    win_length = 1024
    C = np.abs(librosa.stft(x, n_fft = n_fft, hop_length = hop_length, win_length = win_length, window='hann', dtype = np.complex256))
    C = librosa.power_to_db(C**2,ref=np.max)

    if(plot==True):
        plt.figure(figsize=(12, 3))
        plt.title('spectrogram $\mathcal{C}$')
        librosa.display.specshow(C, x_axis='time', y_axis='spectro', cmap='gray_r', hop_length=1024)
        plt.xlabel('Time (frames)'); plt.ylabel('spectro')
        plt.colorbar(); plt.clim([0, 1])
        plt.tight_layout()
    return C


In [ ]:
def get_frame_stats(spectrogram, signal, Fs):
    frames_per_sec = spectrogram.shape[1]/(len(signal)/Fs) # Nbr of frames / length in seconds = frames per second
    frame_duration_sec = 1/frames_per_sec        # frame duration = 1 / frames per second
    return [frames_per_sec, frame_duration_sec]


Convert numpy spectrogram matrix to Pandas dataframe

In [ ]:
def spectrogram_2_dataframe(spectrogram, frame_duration_sec, test_version=False):
    spectrogram = pd.DataFrame(np.transpose(spectrogram), columns=COL_NAMES_FEATURES)

    spectrogram['start'] = np.arange(spectrogram.shape[0]) * frame_duration_sec
    spectrogram['end'] = spectrogram['start'] + frame_duration_sec

    if(test_version == False):

        start_spectrogram = pd.DataFrame(np.random.normal(loc=0, scale=0.01, size=spectrogram.shape[1]),
                                        index=spectrogram.columns).transpose()
        start_spectrogram.iloc[:,-2:] = 0                                
        end_spectrogram = pd.DataFrame(np.random.normal(loc=-1, scale=0.01, size=spectrogram.shape[1]),
                                      index=spectrogram.columns).transpose()
        end_spectrogram.iloc[:,-2:] = spectrogram.iloc[-1]['end']+.01
        spectrogram = start_spectrogram.append(spectrogram, ignore_index=True).append(end_spectrogram, ignore_index=True)

    return spectrogram

In [ ]:
def __get_class_ix(elem, classes_annotation):
    diffs = classes_annotation['start'] - elem
    return diffs[diffs <= 0].index[-1]

In [ ]:
def smooth_classes_by_beat(spectrogram, signal, sr, predicted_col='predicted', n_beats=1):
    _, beats = librosa.beat.beat_track(y=signal, sr=sr)
    beat_times = librosa.frames_to_time(beats, sr=sr)
    beat_times = beat_times * 2 * n_beats
    beat_times[0] = 0
    pcp = spectrogram[['end', predicted_col]].copy()
    pcp['class_cluster'] = np.digitize(pcp['end'], beat_times)
    mode_cluster = pcp.groupby('class_cluster')[predicted_col].agg(lambda x:x.value_counts().index[0])
    pcp['predicted_cluster'] = mode_cluster.loc[pcp['class_cluster']].values
    return pcp['predicted_cluster']

Function for instantiate an object of GMM_HMM class

In [ ]:
def build_gaussian_hmm(initial_state_prob, transition_matrix, mu_array, states_cov_matrices):
    # continuous emission model
    h_markov_model = GMM_HMM(n_components=transition_matrix.shape[0])
    # initial state probability
    h_markov_model.startprob_ = initial_state_prob
    # transition matrix probability
    h_markov_model.transmat_ = transition_matrix.values

    # part of continuous emission probability - multidimensional gaussian
    # mean vector of shape [n_states, n_features]
    h_markov_model.means_ = mu_array
    # array of covariance of shape [n_states, n_features, n_features]
    h_markov_model.covars_ = states_cov_matrices
    return h_markov_model

Calculation of different HMM matrices

In [ ]:
def calc_hmm_matrices(spectrogram, process_silence=True, test_version=False):
    """ Calculates the matrices of a HMM model"""
    # Transition Matrix
    transition_matrix2 = calc_transition_prob_matrix(spectrogram, test_version=test_version)
    # mu and covariance matrices
    mu_array2, states_cov_matrices2 = get_mu_sigma_from_spectro(spectrogram)
    # return filtered_initial_states2, transition_matrix2, mu_array2, states_cov_matrices2
    return  transition_matrix2, mu_array2, states_cov_matrices2

Predictions of HMM model

In [ ]:
def get_predictions(h_markov_model, prediction_dict, song_path, test_version=False):
    # get predictions
    signal, sr, spectrogram = build_spectro(song_path, process_silence=True, test_version=False)
    class_ix_predictions2 = h_markov_model.predict(spectrogram[COL_NAMES_FEATURES])

    spectrogram['predicted'] = get_hmm_predictions(class_ix_predictions2, prediction_dict)
    spectrogram['predicted_cluster'] = smooth_classes_by_beat(spectrogram, signal, sr, predicted_col='predicted', n_beats=1)
    return spectrogram

Convert Numpy matrices of training data to Pandas dataframes

In [ ]:
X_train_df = pd.DataFrame(data = X_train, columns = COL_NAMES_FEATURES )
labels = []
for i in range(Y_train.shape[0]):
  if (Y_train[i][0]==1):
    labels.append('speech')
  elif (Y_train[i][1]==1):
    labels.append('music')
  else:
    labels.append('silence')
    

label = np.array(labels)
Y_train_df = pd.DataFrame(data = label, columns = ['class'] )
final_input = pd.concat([X_train_df, Y_train_df], axis=1, join="inner")  #concatenating the feature df and the labels df
final_input.shape

(315504, 514)

Calculate State, Transition Matrix, Means and Covariance Matrix

In [ ]:
transition_matrix, mu_array, states_cov_matrices = calc_hmm_matrices(final_input, test_version=False)
ix_2_class = {ix_: class_str for ix_,class_str in zip(range(len(mu_array.index.values)),mu_array.index.values)}

In [ ]:
transition_matrix =  transition_matrix.drop(["<START>", "<END>"], axis = 1, inplace = False)
transition_matrix =  transition_matrix.drop([ "<END>"], axis = 0, inplace = False)

In [ ]:
transition_matrix 

sequence_class,music,silence,speech
initial_classes,,,
music,0.989450,0.005048,0.005502
silence,0.003833,0.992683,0.003484
speech,0.004420,0.005092,0.990489


Dimensions of Means and Covariance Matrices

In [ ]:
print(mu_array.shape)
print(states_cov_matrices.shape)

(3, 513)
(3, 513, 513)


Make a Gaussian HMM Model with the previously calculated matrices

In [ ]:
h_markov_model = build_gaussian_hmm(initial_state_probs,transition_matrix,  mu_array, states_cov_matrices)

## Predict


Predictions on the validation set

Validation set contains 170 audio files of 10s each

In [ ]:
output = pd.DataFrame()
validation_folder_name = '/content/drive/MyDrive/Ee603_mlsp/Project/validation'
for audio_filename in os.listdir(validation_folder_name):
  filename = '/content/drive/MyDrive/Ee603_mlsp/Project/validation/'+audio_filename
  x, Fs = librosa.load(filename,sr=16000)
  spectrogram_features = calc_spectrogram(x, Fs, plot=False)
  
  class_ix_predictions2 = h_markov_model.predict(np.moveaxis(spectrogram_features,0,1))
  state_predictions = get_hmm_predictions(class_ix_predictions2, ix_2_class)

  frames_per_sec, frame_duration_sec = get_frame_stats(spectrogram=spectrogram_features, signal=x, Fs=Fs)
  features_df = spectrogram_2_dataframe(spectrogram_features, frame_duration_sec, test_version=True)
  features_df['predicted'] = state_predictions
  classes_simplified = simplify_predicted_classes(features_df)
  classes_simplified['filename'] = audio_filename
  output = pd.concat([output, classes_simplified], axis=0, join="outer")

output = output[['filename', 'start', 'end', 'predicted']]   #reordering the columns
output.to_csv('task1.csv')  #save the output as csv file

Save Model Matrices

In [ ]:
np.save('/content/drive/MyDrive/Ee603_mlsp/Project/hmm_matrices/transition_matrix.npy',transition_matrix)
np.save('/content/drive/MyDrive/Ee603_mlsp/Project/hmm_matrices/ispm.npy', initial_state_probs)
np.save('/content/drive/MyDrive/Ee603_mlsp/Project/hmm_matrices/mu_array.npy', mu_array)
np.save('/content/drive/MyDrive/Ee603_mlsp/Project/hmm_matrices/states_cov_matrices.npy', states_cov_matrices)

#Evaluation



In [ ]:
train_meta = csv_to_meta_container('/content/results_1.csv')

MetaDataContainer :: Class
Items                               : 1888 
Unique
  Files                             : 170 
  Scene labels                      : 0 
  Event labels                      : 3 
  Tags                              : 0 
  Identifiers                       : 0 
  Source labels                     : 0 

Meta data
  Source                  Onset   Offset   Scene             Event             Tags              Identifier   
  --------------------   ------   ------   ---------------   ---------------   ---------------   -----   
  S28.wav                  0.00     1.53   -                 music             -                 -       
  S28.wav                  1.53     1.57   -                 speech            -                 -       
  S28.wav                  1.57     4.98   -                 music             -                 -       
  S28.wav                  4.98     8.47   -                 speech            -                 -       
  S28.wav             

In [ ]:
res = dcase_util.containers.MetaDataContainer(filename=os.path.join(data_storage_path, 'results_sed_hmm.csv'))
for file_name in train_meta.unique_files:
  event_list = train_meta.filter(filename=file_name)
  event_list = event_list.process_events(minimum_event_gap = 0.5)
  event_list = event_list.process_events(minimum_event_length = 0.7)
  print(file_name)
  res += event_list

res = res.filter(event_list = ['speech', 'music'])

res.save('validation_results.csv')
# res.save('temp.csv')

S1016.wav
S1017.wav
S1018.wav
S1019.wav
S1020.wav
S116.wav
S117.wav
S118.wav
S119.wav
S120.wav
S146.wav
S147.wav
S148.wav
S149.wav
S150.wav
S176.wav
S177.wav
S178.wav
S179.wav
S180.wav
S206.wav
S207.wav
S208.wav
S209.wav
S210.wav
S236.wav
S237.wav
S238.wav
S239.wav
S240.wav
S26.wav
S266.wav
S267.wav
S268.wav
S269.wav
S27.wav
S270.wav
S28.wav
S29.wav
S296.wav
S297.wav
S298.wav
S299.wav
S30.wav
S300.wav
S326.wav
S327.wav
S328.wav
S329.wav
S330.wav
S356.wav
S357.wav
S358.wav
S359.wav
S360.wav
S386.wav
S387.wav
S388.wav
S389.wav
S390.wav
S416.wav
S417.wav
S418.wav
S419.wav
S420.wav
S446.wav
S447.wav
S448.wav
S449.wav
S450.wav
S476.wav
S477.wav
S478.wav
S479.wav
S480.wav
S506.wav
S507.wav
S508.wav
S509.wav
S510.wav
S536.wav
S537.wav
S538.wav
S539.wav
S540.wav
S56.wav
S566.wav
S567.wav
S568.wav
S569.wav
S57.wav
S570.wav
S58.wav
S59.wav
S596.wav
S597.wav
S598.wav
S599.wav
S60.wav
S600.wav
S626.wav
S627.wav
S628.wav
S629.wav
S630.wav
S656.wav
S657.wav
S658.wav
S659.wav
S660.wav
S686.wav
S687.w

[{'event_label': 'music',
  'filename': 'S1016.wav',
  'offset': 7.092651757,
  'onset': 5.015974441},
 {'event_label': 'music',
  'filename': 'S1016.wav',
  'offset': 8.370607029,
  'onset': 7.635782748},
 {'event_label': 'music',
  'filename': 'S1016.wav',
  'offset': 10.0,
  'onset': 9.009584665},
 {'event_label': 'speech',
  'filename': 'S1016.wav',
  'offset': 9.584664537,
  'onset': 7.092651757},
 {'event_label': 'music',
  'filename': 'S1017.wav',
  'offset': 8.785942492,
  'onset': 5.367412141},
 {'event_label': 'speech',
  'filename': 'S1017.wav',
  'offset': 6.677316294,
  'onset': 5.015974441},
 {'event_label': 'speech',
  'filename': 'S1017.wav',
  'offset': 9.968051118,
  'onset': 8.785942492},
 {'event_label': 'music',
  'filename': 'S1018.wav',
  'offset': 10.0,
  'onset': 5.015974441},
 {'event_label': 'music',
  'filename': 'S1019.wav',
  'offset': 10.0,
  'onset': 5.015974441},
 {'event_label': 'music',
  'filename': 'S1020.wav',
  'offset': 10.0,
  'onset': 5.0159744

## Preparing data for evaluation

Prepare reference data and estimated to have filenames in uniform format:

In [ ]:
validation_data = csv_to_meta_container('/content/drive/MyDrive/Ee603_mlsp/Project/validation.csv')

MetaDataContainer :: Class
Items                               : 360 
Unique
  Files                             : 170 
  Scene labels                      : 0 
  Event labels                      : 2 
  Tags                              : 0 
  Identifiers                       : 0 
  Source labels                     : 0 

Meta data
  Source                  Onset   Offset   Scene             Event             Tags              Identifier   
  --------------------   ------   ------   ---------------   ---------------   ---------------   -----   
  S26                      0.00     5.00   -                 music             -                 -       
  S26                      5.00    10.00   -                 speech            -                 -       
  S27                      0.00     5.00   -                 music             -                 -       
  S27                      5.00    10.00   -                 speech            -                 -       
  S28                  

In [ ]:
reference_event_list = validation_data
estimated_event_list = dcase_util.containers.MetaDataContainer(
    filename=os.path.join('/content/', 'validation_results.csv')
).load()

for item_id, item in enumerate(estimated_event_list):
    estimated_event_list[item_id]['filename'] = item.filename.split('.')[0]    

In [ ]:
print(estimated_event_list)

MetaDataContainer :: Class
Filename                            : /content/validation_results.csv 
Items                               : 403 
Unique
  Files                             : 170 
  Scene labels                      : 0 
  Event labels                      : 2 
  Tags                              : 0 
  Identifiers                       : 0 
  Source labels                     : 0 

Meta data
  Source                  Onset   Offset   Scene             Event             Tags              Identifier   
  --------------------   ------   ------   ---------------   ---------------   ---------------   -----   
  S1016                    5.02     7.09   -                 music             -                 -       
  S1016                    7.64     8.37   -                 music             -                 -       
  S1016                    9.01    10.00   -                 music             -                 -       
  S1016                    7.09     9.58   -              

In [ ]:
print(reference_event_list)

MetaDataContainer :: Class
Items                               : 360 
Unique
  Files                             : 170 
  Scene labels                      : 0 
  Event labels                      : 2 
  Tags                              : 0 
  Identifiers                       : 0 
  Source labels                     : 0 

Meta data
  Source                  Onset   Offset   Scene             Event             Tags              Identifier   
  --------------------   ------   ------   ---------------   ---------------   ---------------   -----   
  S26                      0.00     5.00   -                 music             -                 -       
  S26                      5.00    10.00   -                 speech            -                 -       
  S27                      0.00     5.00   -                 music             -                 -       
  S27                      5.00    10.00   -                 speech            -                 -       
  S28                  

In [ ]:
# Initialize evaluator with list of event labels to be evaluated and segment length
evaluator = sed_eval.sound_event.SegmentBasedMetrics(
    event_label_list=['speech', 'music'], 
    time_resolution=0.2                    # 1 second segments
)
# Loop file by file and accumulate intermediate statistics
for filename in reference_event_list.unique_files:
    evaluator.evaluate(
        reference_event_list=reference_event_list.filter(filename=filename),
        estimated_event_list=estimated_event_list.filter(filename=filename)
    ) 
metrics = evaluator.results_overall_metrics()

## Metric values

In [ ]:
log.table(
    column_headers=['Metric', 'Value'],
    cell_data=[
        [
          '<strong>F-score</strong>',
          'Precision',
          'Recall',
          '<strong>Error rate</strong>',
            'Substitutions',
            'Deletions',
            'Insertions'
        ],
        [
            metrics['f_measure']['f_measure']*100.0,
            metrics['f_measure']['precision']*100.0,
            metrics['f_measure']['recall']*100.0,
            metrics['error_rate']['error_rate'],
            metrics['error_rate']['substitution_rate'],
            metrics['error_rate']['deletion_rate'],
            metrics['error_rate']['insertion_rate'],
        ]
    ],
    row_separators=[3],
    scaling=130
)

Metric,Value
F-score,90.74
Precision,84.77
Recall,97.62
Error rate,0.18
Substitutions,0.02
Deletions,0.00
Insertions,0.15


## Class-wise metrics

In [ ]:
class_metrics = evaluator.results_class_wise_metrics()
Nref = []
Nsys = []
Fscore=[]
for event_label in list(class_metrics.keys()):
    Nref.append(class_metrics[event_label]['count']['Nref'])
    Nsys.append(class_metrics[event_label]['count']['Nsys'])
    Fscore.append(class_metrics[event_label]['f_measure']['f_measure']*100.0)
log.table(
    column_headers=['Event', 'Nref', 'Nsys', 'Fscore'],
    cell_data=[
        list(class_metrics.keys()),Nref,Nsys,Fscore
    ],
    column_types=['str25', 'int', 'int', 'float2'],
    column_separators=[0,2],
    scaling=130
)

Event,Nref,Nsys,Fscore
speech,2625,3132,90.74
music,2375,2626,90.74


Output directly from `sed_eval` evaluator:

In [ ]:
print(evaluator)    

Segment based metrics
  Evaluated length                  : 1540.55 sec
  Evaluated files                   : 170 
  Segment length                    : 200.00 ms

  Overall metrics (micro-average)
  F-measure
    F-measure (F1)                  : 90.74 %
    Precision                       : 84.77 %
    Recall                          : 97.62 %
  Error rate
    Error rate (ER)                 : 0.18 
    Substitution rate               : 0.02 
    Deletion rate                   : 0.00 
    Insertion rate                  : 0.15 
  Accuracy
    Sensitivity                     : 97.62 %
    Specificity                     : 91.60 %
    Balanced accuracy               : 94.61 %
    Accuracy                        : 93.55 %

  Class-wise average metrics (macro-average)
  F-measure
    F-measure (F1)                  : 90.74 %
    Precision                       : 84.90 %
    Recall                          : 97.52 %
  Error rate
    Error rate (ER)                 : 0.20 
    Deletion ra